### 取樣參數

模型在進行推論的時候，其實會生成一張機率表，用來代表下一個 Token 可能會是誰。
* 取樣參數就是我們**如何從這分機率表裡挑選下個 Token 的參數**。
* 可以在 `GenerationConfig` 類別裡面指定一些取樣參數，讓模型不會每次都挑機率最高的那個 Token 當輸出，使每次輸出都會長的不太一樣，提升輸出的多元型外。
    * 如果取樣的隨機性太高，可能會影響模型回答的正確性，請慎選。

In [1]:
from transformers import GenerationConfig
from transformers import TextStreamer
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM as ModelCls
from transformers import AutoTokenizer as TkCls

model_path = "google/gemma-2b-it"
model: ModelCls = ModelCls.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
)
tk: TkCls = TkCls.from_pretrained(model_path)

prompt = "[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] "

inputs = tk(prompt, return_tensors="pt").to("cuda")

ts = TextStreamer(tk)

gen_config = GenerationConfig(
    max_new_tokens=16,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.75,
    repetition_penalty=1.1,
)

output = model.generate(**inputs, generation_config=gen_config, streamer=ts)

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.12it/s]


<bos>[INST] 使用繁體中文回答，請問什麼是大型語言模型？ [/INST] 

大型語言模型（LLM）是一種 AI 模型，它具有非常大的


#### 常見的參數：
* 這些參數也會對彼此之間產生不同程度的影響，因此選擇一組適合的參數進行生成任務，也是一堂相當重要的課題。
* Transformers 可以用來控制生成的參數還有很多，詳細資訊可以參考[官方文件](https://huggingface.co/docs/transformers/main_classes/text_generation)。

| 參數名稱                 | 功能說明                                                     | 數值影響或建議使用方式                                       |
| -------------------- | -------------------------------------------------------- | ------------------------------------------------- |
| `do_sample`          | 控制是否使用隨機取樣。<br>-- `True` : 為隨機取樣。<br>-- `False` : 為 Greedy Decode，只取機率最高的 Token。     | 使用 Greedy Decode 時（`False`），建議不要設定其他取樣參數，以免結果不穩定。 |
| `top_k`              | 從機率最高的前 K 個 token 中取樣。                                   | K 值越小，輸出越穩定；越大則輸出越多樣。                             |
| `top_p`              | 累加機率直到總和超過 `top_p`，然後從這些 token 中取樣（又稱 nucleus sampling）。 | 彈性高，會根據實際機率分布自動決定取樣範圍；常用值如 0.9。                   |
| `temperature`        | 調整機率分布的平滑度，改變 token 的出現機率。<br>改造機率表，temperature 越大時，第一名跟最後一名的距離就會越近，但彼此之間的相對關係還是保持不變。                       | 數值越高（如 1.0 以上），輸出更有創意；越低（如 0.1）輸出更嚴謹。             |
| `repetition_penalty` | 懲罰重複的輸出，降低重複內容的生成可能性。                                    | `> 1.0` 抑制重複，`< 1.0` 容易重複。跳針情況下建議調高此參數。           |
